<a href="https://colab.research.google.com/github/AlexandrPinevich/Domino_to_1c_export/blob/main/Domino_1C_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Эта штука должна парсить csv и исходники и рожать из них данные на вход в 1С для ввода начальных остатков. Файлы берутся из гугл диска. Для целей консистентности данных сильно желательно все файлы выгружать из Домина одним днем и одним пакетом

In [102]:
import pandas as pd            #БЕЗ ЭТОГО РАБОТАТЬ НЕ БУДЕТ!!!
import numpy as np
from datetime import datetime  #НАДО ЗАПУСКАТЬ ПЕРЕД ОБРАБОТКОЙ
import pytz                    #ВООБЩЕ ВСЕ ПО ОЧЕРЕДИ НАДО ЗАПУСКАТЬ
!pip install xlsxwriter        #ИНАЧЕ ЭКСЕЛЬ НЕ ПИШЕТСЯ ИЗ-ЗА КОДИРОВКИ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
# Пути
input_path = '/content/drive/MyDrive/Colab Notebooks/Input/'
output_path = '/content/drive/MyDrive/Colab Notebooks/Output/'
time_now = '_' + (datetime.now(pytz.timezone('Europe/Moscow'))\
          .strftime("%Y_%b_%d %H.%M.%S"))
to_xlsx = '.xlsx'
to_csv = '.csv'

## Блок считывания файлов. Папка Input








In [104]:
# Это читает ИСХОДНИК текущий партии
# предобработанный, но без ручной колонки партий
file_name = '!!!!Исходник текущий партии.xlsx'
file_name = ''.join((input_path, file_name))
batch=pd.read_excel(file_name,header = 4, skipfooter = 1, keep_default_na=True\
                    , dtype={'ТОВАР': object}, sheet_name='gr_post')
batch.rename(columns={'ТОВАР': 'Партия'}, inplace=True)
batch.rename(columns={'ЕИ': 'ЕИпартии'}, inplace=True)
batch.insert(0,'ТОВАР','')
batch['ТОВАР'] = batch['Партия'].astype('Float64').astype('Int64')
batch.insert(2,'Номенклатура','')
batch['Номенклатура'] = 'ДМ-' + batch['ТОВАР'].apply('{:0>8}'.format)\
                                                             .astype(str)
batch.dropna(subset=['ИСХ_КОЛИЧ'], inplace = True)
# Дропает строки без остатка насовсем

In [105]:
# Это читает ИСХОДНИК текущий уже экселем предобработанный
file_name = '!!!!Исходник текущий.xlsx'
file_name = ''.join((input_path, file_name))
df2=pd.read_excel(file_name, header = 4, skipfooter = 1, keep_default_na=True\
                  , sheet_name='gr_post')
df2.insert(1,'Номенклатура','')
df2['Номенклатура'] = 'ДМ-' + df2['ТОВАР'].apply('{:0>8}'.format).astype(str)

In [106]:
# Это читает файл ###Справочники
file_name = '###Справочники.xlsx'
file_name = ''.join((input_path, file_name))
unitsDf=pd.read_excel(file_name, sheet_name='units', keep_default_na=True\
                          , dtype={'КодЕд': object})
managersDf=pd.read_excel(file_name, sheet_name='managers', keep_default_na=True)
warehousesDf=pd.read_excel(file_name, sheet_name='warehouses'\
                          , keep_default_na=True)
batch_statusDf=pd.read_excel(file_name, sheet_name='batch_status'\
                          , keep_default_na=True)
suppliersDf=pd.read_excel(file_name, sheet_name='suppliers'\
                          , keep_default_na=True\
                          , dtype={'Поставщик_Номер_в_Домино': object})

In [107]:
# Это читает BARCODES.CSV сырой, как выпал из отчета Домино
# keep_default_na = 1/0 ПЕРЕКЛЮЧАТЕЛЬ ВЫВОДА NaN
file_name = 'barcodes.csv'
file_name = ''.join((input_path, file_name))
header_list = ['ТОВАР','Штрихкод','ТОВАР2','НАИМЕНОВАНИЕ_ДЛИННОЕ',
          'АРТИКУЛ_ДЛИННЫЙ','GrNum','Group','SrgNum','SubGroupeAsIs',
          'supplierCode','supplier','countryCode','country']

df1 = pd.read_csv(file_name, delimiter = ';', header=None, names = header_list,
            encoding='Windows-1251')
#,header=None, header=None, iterator = False, index_col = False, 
# , engine='c''pyarrow', nrows=100 keep_default_na=True,, low_memory=False

df1.insert(1,'Номенклатура','')
df1['Номенклатура'] = 'ДМ-' + df1['ТОВАР'].apply('{:0>8}'.format).astype(str)
df1['SubGroupe'] = df1['SubGroupeAsIs'].str.upper()

In [108]:
# Это читает SKU из 1С нужно чтобы только новые карточки можно было добавить
# или страрые удалить
file_name = 'SKU from 1C.xlsx'
file_name = ''.join((input_path, file_name))
from1c = pd.read_excel(file_name, sheet_name='TDSheet')
from1c['Номенклатура'] ='ДМ-' + from1c['Код Домино'].apply('{:0>8}'.format)\
    .astype(str)
from1c['Менеджер'] = from1c['Ссылка.Менеджер ("Общие")']
from1c['Мен1С'] = from1c['Ссылка.Менеджер ("Общие")']

In [109]:
# Это читает Категории из 1С 
file_name = 'Категории из 1С.xlsx'
file_name = ''.join((input_path, file_name))
catalog1c=pd.read_excel(file_name,sheet_name='TDSheet')
catalog1c['SubGroupe'] = catalog1c['Подгруппа']

In [110]:
# Это читает Штрихкоды из 1С 
file_name = 'Штрихкода из 1с.xlsx'
file_name = ''.join((input_path, file_name))
BarFrom1c=pd.read_excel(file_name,sheet_name='TDSheet')
BarFrom1c['Номенклатура'] = BarFrom1c['Номенклатура.Код']

In [111]:
# Это читает Цены из 1С 
file_name = 'Цены розница из 1с.xlsx'
file_name = ''.join((input_path, file_name))
PriceFrom1c=pd.read_excel(file_name,sheet_name='TDSheet')
PriceFrom1c.rename(columns={'Номенклатура': 'Наименование'
                          ,'Код' : 'Номенклатура'
                          ,'Ед.' : 'Единица измерения'                            
                          }, inplace=True)

In [112]:
# Это читает Партии из 1С
file_name = 'Партии из 1с.xlsx'
file_name = ''.join((input_path, file_name))
batch_from1c_df = pd.read_excel(file_name,sheet_name='TDSheet',\
                               dtype={'Ссылка': object})
batch_from1c_df['Наименование']  = batch_from1c_df['Ссылка']
batch_from1c_df['Партия']  = batch_from1c_df['Ссылка']
batch_from1c_df['Номенклатура']  = batch_from1c_df['Номенклатура.Код']

In [113]:
# Это читает 700ку
# Пофиксить возврат ДЖБА, косяки с партиями МК, строки портит обоим
file_name = 'Семисотка текущая.xlsx'
file_name = ''.join((input_path, file_name))
MoneyInputDf=pd.read_excel(file_name, header = 5, skipfooter = 2,
                        usecols = ('A:L'), keep_default_na=True, sheet_name=0)

## Блок обработки

In [114]:
# Фрейм BarNoDup Убирает дубликаты строк из df1 - BARCODES.CSV по коду,
# нужно для корректного джойна с исходником
BarNoDup = df1.copy(deep=True).drop_duplicates(subset=['ТОВАР'])

In [115]:
# Фрейм BarcodeExport Забираем из таблицы с штрихкодами 2 колонки
# в новый датафрейм BarcodeExport для последующей выгрузки
BarcodeExport = df1[['Номенклатура', 'Штрихкод']].copy(deep=True)
BarcodeExport = BarcodeExport.merge(df2['Номенклатура'], how='inner')
# Оставить только совпавшие c исходником строки,
# Нюансик, строки без штрихкода дропаются тоже how='inner'                                                                   

In [116]:
# Фрейм skuDf копирует исходник чтобы избежать возможного конфликта
# последовательности загрузки, в нем же добавляем колонки, кот. нужны на импорт
# а не надо здесь ешё наличие штрихкода в домине проверить к этим карточкам?
skuDf = df2.copy(deep=True)
skuDf = skuDf\
    .merge(BarNoDup[['НАИМЕНОВАНИЕ_ДЛИННОЕ','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['АРТИКУЛ_ДЛИННЫЙ','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['SubGroupe','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(unitsDf[['КодЕд','ЕИ']], on='ЕИ', how='left')\
    .merge(suppliersDf[['Поставщик_Имя','ПОСТАВЩИК']]\
           , on='ПОСТАВЩИК', how='left')\
    .merge(managersDf[['Мен1С','МЕНЕДЖЕР']], on='МЕНЕДЖЕР', how='left')\
    .merge(catalog1c, left_on='SubGroupe', right_on='SubGroupe', how='left')

skuDf.insert(36,'Наименование полное','')
skuDf['Наименование полное'] = skuDf['НАИМЕНОВАНИЕ_ДЛИННОЕ']
skuDf['Направление деятельности'] = 'Основное направление'
skuDf['Использовать партии'] = 'Да'
skuDf['Обязательное заполнение партий'] = 'Да'
skuDf['Нижняя граница остатков'] = skuDf['МИН'].abs()
skuDf['Верхняя граница остатков'] = skuDf['УП'].abs()
skuDf['Цена'] = skuDf['ЦЕНА(ПР)']

In [117]:
# Фрейм SKUexport Забираем из skuDf нужные колонки и переименовываем 
# для последующей выгрузки в файл
SKUexport =skuDf[['Номенклатура', 'НАИМЕНОВАНИЕ_ДЛИННОЕ',
       'Наименование полное','АРТИКУЛ_ДЛИННЫЙ', 'SubGroupe', 'КодЕд',
       'Поставщик_Имя', 'Тип номенклатуры по умолчанию', 'Вид ставки НДС',
       'Способ списания', 'Счет учета запасов', 'Счет учета затрат',
       'Способ пополнения', 'Вид маркировки', 'Вид маркируемой продукции ИС МП',
       'Обувная продукция', 'Признак предмета расчета',
       'Направление деятельности', 'Использовать партии',
       'Обязательное заполнение партий', 'Нижняя граница остатков',
       'Верхняя граница остатков']].copy(deep=True)
#  'Ценовая группа', # в будущем может пригодиться
SKUexport.rename(columns={
                 'НАИМЕНОВАНИЕ_ДЛИННОЕ' : 'Наименование'
                ,'АРТИКУЛ_ДЛИННЫЙ' : 'Артикул'
                ,'SubGroupe' : 'Категория'
                ,'Тип номенклатуры по умолчанию' : 'ТипНоменклатуры'
                ,'Способ списания' : 'МетодОценки'
                ,'КодЕд' : 'Единица измерения'
                ,'Обязательное заполнение партий' : 'ПроверятьЗаполнениеПартий'
                ,'Вид маркируемой продукции ИС МП' : 'ВидПродукцииИС'
                ,'Поставщик_Имя' : 'Поставщик'
                        },inplace=True)
SKUexport = SKUexport.copy(deep=True)

In [118]:
# Фрейм SKUexportAll (вся номенклатура из исходника)
# Фрейм SKUexportNew (номенклатуры нет в 1с, но есть в исходнике)
# Фрейм SKUexportOld (номенклатуры нет в исходнике, но есть в 1с)
SKUexportAll = SKUexport.copy(deep=True)
SKUexportAll.rename(columns={'Номенклатура': 'Код'}, inplace=True)

SKUexportNew = SKUexport.copy(deep=True)
SKUexportNew = SKUexportNew\
  .merge(from1c[['Код','Номенклатура']], on='Номенклатура', how='left')
SKUexportNew = SKUexportNew[SKUexportNew['Код'].isna()] 
SKUexportNew.drop(columns=['Код'], inplace= True) # все равно пустая
# тут NaN по коду это строки с новыми карточками, вытащим их как раз
SKUexportNew.rename(columns={'Номенклатура': 'Код'}, inplace=True)
 
SKUexportOld = SKUexport.copy(deep=True)
SKUexportOld = SKUexport\
  .merge(from1c[['Код','Номенклатура']], on='Номенклатура', how='right')\
  .merge(BarNoDup[['SubGroupe','Номенклатура']], on='Номенклатура',how='left')\

SKUexportOld['ТОВАР'] = SKUexportOld['Код'].str[3:].astype('Int64') #код домино
SKUexportOld = SKUexportOld[SKUexportOld['SubGroupe'].notna()]#оставить тестовый
SKUexportOld = SKUexportOld[SKUexportOld['Наименование'].isna()]
SKUexportOld = SKUexportOld[['Номенклатура','SubGroupe']]
SKUexportOld = SKUexportOld\
    .merge(from1c[['Недействительна','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(from1c[['Категория','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(from1c[['Наименование','Номенклатура']]\
           , on='Номенклатура', how='left')

SKUexportOld.rename(columns={'Номенклатура': 'Код'}, inplace=True)
# Все коды из 1с, тут NaN по наименованию это то, что в исходнике не нашлось
# выдает просто список кодов. Не убить тестовые раз. Можно достать из
# баркодов новую группу и заменить, не удалять их 
# merge изначально не работал из-за разного столбца "номенклатура"
# из домино и из 1с, кодировка?

# Переименовать Номенклатуру в Код для 1с пришлось после, костыль

In [149]:
# Фрейм SKU Update Все строки, где не совпало 1с и Домино
SkuUpdate = from1c[['Код Домино', 'Номенклатура', 'Наименование'
       ,'Артикул','Категория', 'Поставщик', 'Ссылка.Менеджер ("Общие")'
       , 'Недействительна']].copy(deep=True)
SkuUpdate = SkuUpdate\
  .merge(PriceFrom1c[['Цена','Номенклатура']], on='Номенклатура', how='left')\
  .merge(BarNoDup[['Штрихкод','Номенклатура']], on='Номенклатура', how='left')

###############################################################################
SkuUpdate = SkuUpdate\
    .merge(SKUexport[['Наименование','Номенклатура']]\
           , on='Номенклатура', how='left', indicator=True)
SkuUpdate.rename(columns={'_merge': 'merge1'}, inplace=True)
SkuUpdate['eq1'] = SkuUpdate['Наименование_y'].str.strip().str.lower()\
                == SkuUpdate['Наименование_x'].str.strip().str.lower()

SkuUpdate = SkuUpdate\
    .merge(SKUexport[['Артикул','Номенклатура']]\
           , on='Номенклатура', how='left', indicator=True)
SkuUpdate.rename(columns={'_merge': 'merge2'}, inplace=True)
SkuUpdate['eq2'] = SkuUpdate['Артикул_y'].str.strip().str.lower()\
                == SkuUpdate['Артикул_x'].str.strip().str.lower()

SkuUpdate = SkuUpdate\
    .merge(SKUexport[['Категория','Номенклатура']]\
           , on='Номенклатура', how='left', indicator=True)
SkuUpdate.rename(columns={'_merge': 'merge3'}, inplace=True)
SkuUpdate['eq3'] = SkuUpdate['Категория_y'].str.strip().str.lower()\
                == SkuUpdate['Категория_x'].str.strip().str.lower()

SkuUpdate = SkuUpdate\
    .merge(SKUexport[['Поставщик','Номенклатура']]\
           , on='Номенклатура', how='left', indicator=True)
SkuUpdate.rename(columns={'_merge': 'merge4'}, inplace=True)
SkuUpdate['eq4'] = SkuUpdate['Категория_y'].str.strip().str.lower()\
                == SkuUpdate['Категория_x'].str.strip().str.lower()

SkuUpdate = SkuUpdate\
    .merge(skuDf[['Мен1С','Номенклатура']]\
           , on='Номенклатура', how='left', indicator=True)
SkuUpdate.rename(columns={'_merge': 'merge5'}, inplace=True)
SkuUpdate['eq5']\
            = SkuUpdate['Ссылка.Менеджер ("Общие")'].str.strip().str.lower()\
           == SkuUpdate['Мен1С'].str.strip().str.lower()

SkuUpdate = SkuUpdate\
    .merge(skuDf[['Цена','Номенклатура']]\
           , on='Номенклатура', how='left', indicator=True)
SkuUpdate.rename(columns={'_merge': 'merge6'}, inplace=True)
SkuUpdate['eq6'] = SkuUpdate['Цена_y'] == SkuUpdate['Цена_x']

# Все строки, где не совпало 1с и Домино
SkuUpdate['eqAll'] = SkuUpdate['eq1'] * SkuUpdate['eq2'] * SkuUpdate['eq3']\
    * SkuUpdate['eq4'] * SkuUpdate['eq5'] * SkuUpdate['eq6']

###############################################################################
# Оставляем только совпавшие с актуальной базой Домино
SkuUpdate = SkuUpdate.query("merge1 == 'both'")
# Оставляем только несовпавшие значения по колонкам с 1С
SkuUpdate = SkuUpdate.query("eqAll == False")
# Оставляем только нужные колонки
SkuUpdate = SkuUpdate[['Номенклатура', 'Штрихкод', 'Наименование_y'
    , 'Артикул_y', 'Категория_y', 'Поставщик_y', 'Мен1С', 'Цена_y']]

SkuUpdate.rename(columns={
                 'Наименование_y' : 'Наименование'
                ,'Артикул_y' : 'Артикул'
                ,'Категория_y' : 'Категория'
                ,'Поставщик_y' : 'Поставщик'
                ,'Мен1С' : ' Менеджер'
                ,'Цена_y' : 'Цена'
                        },inplace=True)   
SkuUpdate.insert(loc=3, column='Наименование полное'
                       ,value = SkuUpdate['Наименование'])


In [120]:
# Фрейм BarcodeExportNew только штрихкода к новым карточкам
# Проверять наличие уже их в 1с тогда надо бы сразу
BarcodeExportNew = df1[['Номенклатура', 'Штрихкод']].copy(deep=True)
BarcodeExportNew.rename(columns={'Номенклатура': 'Код'}, inplace=True)
BarcodeExportNew = BarcodeExportNew.merge(SKUexportNew['Код']\
                                          , how='inner')
BarcodeExportNew.rename(columns={'Код': 'Номенклатура'}, inplace=True)
# Оставить только совпавшие SKUexportNew строки, новый товар
# Нюансик, строки без штрихкода дропаются тоже how='inner'                                                                   

In [121]:
# Фрейм BarcodeExportMissed таких шк нет в 1с, а коды есть (у живых SKU)
BarcodeExportMissed = BarcodeExport.copy(deep=True)
BarcodeExportMissed = BarcodeExportMissed.merge(BarFrom1c['Штрихкод']\
                                         ,how='left', indicator=True)\
                                         .query("_merge == 'left_only'")\
                                         .drop('_merge', axis=1)

In [122]:
# Фрейм BarcodeExportWasted связка код штрихкод 1с, не совпадает с Домино
# А также такие штрихкода есть в 1с, а в домино вообще их нет.
# Сюда тестовые попадают тоже, не грохнуть их случайно
BarcodeExportWasted = BarFrom1c[['Номенклатура', 'Штрихкод'
    ,'Номенклатура.Наименование','Номенклатура.Категория']].copy(deep=True)
BarcodeExportWasted = BarcodeExportWasted\
  .merge(df1[['Номенклатура','Штрихкод']], on='Штрихкод', how='left')
BarcodeExportWasted['Flag']\
     = BarcodeExportWasted['Номенклатура_x']\
    == BarcodeExportWasted['Номенклатура_y']
BarcodeExportWasted = BarcodeExportWasted[~BarcodeExportWasted['Flag']]
BarcodeExportWasted = BarcodeExportWasted\
    [~BarcodeExportWasted['Номенклатура.Наименование'].str.contains('!ТЕСТ')]
BarcodeExportWasted = BarcodeExportWasted[['Штрихкод']]

In [123]:
# Фрейм PriceExport под штатную загрузку розничной цены и менеджера
PriceExport\
    = skuDf[['Номенклатура', 'Штрихкод', 'ЦЕНА(ПР)','Мен1С']].copy(deep=True)
PriceExport['Штрихкод'] =  PriceExport['Штрихкод'].astype(str)
PriceExport.rename(columns={'ЦЕНА(ПР)': 'Цена'}, inplace=True)
PriceExport.rename(columns={'Мен1С': 'Менеджер'}, inplace=True)
PriceExport['Вид цен'] = 'Розница'
PriceExport['Период'] = '31.03.2023'

In [124]:
# Фрейм batchExport партии номенклатуры все где есть остаток в исходнике
batchExport = batch[['Номенклатура', 'Партия',
                     'ПОСТАВЩИК', 'ДОГ']].copy(deep=True)
batchExport = batchExport\
    .merge(batch_statusDf[['СтатусОплаты1С','ДОГ']], on='ДОГ', how='left')\
    .merge(suppliersDf[['Поставщик_Статус_НДС','ПОСТАВЩИК']]\
                        , on='ПОСТАВЩИК', how='left')\
    .merge(suppliersDf[['Поставщик_Имя','ПОСТАВЩИК']]\
                        , on='ПОСТАВЩИК', how='left')
batchExport['Наименование'] = batchExport['Партия'].astype(str)
batchExport['Статус'] = batchExport['СтатусОплаты1С']
batchExport['Владелец партии'] =batchExport['Поставщик_Имя']
batchExport['Налогообложение'] =batchExport['Поставщик_Статус_НДС']

batchExport = batchExport[['Номенклатура','Наименование','Статус' 
                            ,'Владелец партии','Налогообложение',]]

In [125]:
# Фрейм batchExportNewDelete batchExportNewInsert
# Новые одним потоком добавить не удается, оно старые перезаписывает
# как выход, удалить перезаписанные, потом перезалить по этим базовым
batchExportNew = batchExport.copy(deep=True)
batchExportNew = batchExportNew.merge(batch_from1c_df['Наименование']\
                                      ,how='left', indicator=True)\
                                      .query("_merge == 'left_only'")\
                                      .drop('_merge', axis=1)
###############################################################################

# Сначала удаляем партии по этим базовым 
# ИР, цикл в несколько итераций, ключ поиска по базовым карточкам                                   
batchExportNewDelete =  batchExportNew.copy(deep=True)
batchExportNewDelete = batchExportNewDelete\
                     .drop_duplicates(subset=['Номенклатура'])
batchExportNewDelete = batchExportNewDelete\
                     .merge(batch_from1c_df[['Наименование','Номенклатура']]\
                     , on='Номенклатура', how='left')
batchExportNewDelete.dropna(subset=['Наименование_y'], inplace = True)    
batchExportNewDelete = batchExportNewDelete[['Номенклатура','Наименование_y']] 
batchExportNewDelete\
    .rename(columns={'Наименование_y': 'Наименование'}, inplace=True)

###############################################################################

# Потом вгружаем все партии по этим базовым, галку ОБНОВЛЯТЬ снимаем!, ток новые
batchExportNewInsert =  batchExportNew.copy(deep=True)
batchExportNewInsert = batchExportNewInsert\
                     .drop_duplicates(subset=['Номенклатура'])
batchExportNewInsert = batchExportNewInsert\
                     .merge(batchExport[['Наименование','Номенклатура']]\
                     , on='Номенклатура', how='left')
batchExportNewInsert = batchExportNewInsert[['Номенклатура','Наименование_y'
                             , 'Статус', 'Владелец партии','Налогообложение',]]
batchExportNewInsert\
    .rename(columns={'Наименование_y': 'Наименование'}, inplace=True)    

###############################################################################

# Партии, которых есть в 1с, а в домине уже остатка нет

batchExportOldDelete = batch_from1c_df.copy(deep=True)     
batchExportOldDelete = batchExportOldDelete\
                      .merge(batchExport['Наименование']\
                      ,how='left', indicator=True)\
                      .query("_merge == 'left_only'")\
                      .drop('_merge', axis=1)   
batchExportOldDelete = batchExportOldDelete[['Номенклатура','Наименование']]               

In [126]:
# Фрейм bdfExport остатки по партиям
# Комиссия отдельно сделать
# Ошибка если единицы из домина не по списку приходят, case censitive в.т.ч
# Лечить в Домине не дожидаясь перитонита
# в Домине не лечится, возьмем из базовых из исходника
# Допилить проверку ошибок
bdf = batch.copy(deep=True)
bdf['Партия'] = bdf['Партия'].astype(str)

input_rows = bdf.shape[0]
input_qtt = bdf['ИСХ_КОЛИЧ'].sum().round(2)
input_sum = bdf['СебестРуб.3'].sum().round(2)

bdf = bdf\
    .merge(df2[['ЕИ','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['Штрихкод','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['SubGroupe','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(suppliersDf[['Поставщик_Имя','ПОСТАВЩИК']]\
           , on='ПОСТАВЩИК', how='left')\
    .merge(batch_statusDf[['ВидОперации','ДОГ']]\
           , on='ДОГ', how='left')\
    .merge(unitsDf[['КодЕд','ЕИ']], on='ЕИ', how='left')\
    .merge(unitsDf[['1сКрЕИ','ЕИ']], on='ЕИ', how='left')   

bdf = bdf.sort_values(by=['ВидОперации'])
bdf['Контрагент'] = bdf['Поставщик_Имя']
bdf['Единица измерения'] = bdf['1сКрЕИ']

bdf['Договор'] = 'Основной договор'   
bdf[['1эт остаток', '2эт остаток','скл1 остаток', 'б/н остаток',
           'брак остаток', 'ИСХ_КОЛИЧ','СебестРуб.3']] = bdf[['1эт остаток', 
           '2эт остаток','скл1 остаток', 'б/н остаток',
           'брак остаток', 'ИСХ_КОЛИЧ','СебестРуб.3']].fillna(0)    

bdf['хозу остаток'] =  bdf['ИСХ_КОЛИЧ'] - bdf['1эт остаток']\
                     - bdf['2эт остаток'] - bdf['скл1 остаток']\
                     - bdf['б/н остаток'] - bdf['брак остаток']  
bdf['Себестоимость'] = bdf['СебестРуб.3'] / bdf['ИСХ_КОЛИЧ']

serv_rows = bdf[bdf['ГРУППА'] == '!!УСЛУГИ'].index.value_counts().sum()
serv_qtt = bdf[bdf['ГРУППА'] == '!!УСЛУГИ']['ИСХ_КОЛИЧ'].sum().round(2) 
serv_sum = bdf[bdf['ГРУППА'] == '!!УСЛУГИ']['СебестРуб.3'].sum().round(2)
# У услуг нет остатка

bdf.drop(bdf[(bdf['ГРУППА'] == '!!УСЛУГИ')].index, inplace=True)

# тут мы сцепляем остатки по подразделениям в одну таблицу
batchSum = pd.DataFrame()
storage_domino = warehousesDf['Отдел'].values
storage_1c = warehousesDf['СтруктурнаяЕдиница'].values
i = 0
z = 0
for x in storage_domino:
    # print('step ', i) # отладочное
    # print('x=',x)
    # print(storage_1c[i])
    bdfTmp = bdf[bdf[x] != 0].copy()
    bdfTmp['СтруктурнаяЕдиница'] = storage_1c[i] #Склад 1c
    bdfTmp['Количество'] = bdfTmp[x]
    bdfTmp['Сумма'] = bdfTmp['Количество'] * bdfTmp['Себестоимость']
    z = z + bdfTmp.shape[0]
    # print(bdfTmp.shape[0], 'rows')
    # print(z, 'rowsSum')
    batchSum = pd.concat([batchSum, bdfTmp], ignore_index= True)
    i = i + 1
##############################################################################
bdfExportAll = batchSum[['Номенклатура', 'Партия', 'Количество',
                      'Единица измерения', 'Сумма',
                      'СтруктурнаяЕдиница', 'Контрагент',  # 'Себестоимость',
                      'ВидОперации', 'Договор'              
                     ]].copy()

output_rows = batchSum.shape[0]
output_qtt = batchSum['Количество'].sum().round(2)
output_sum = batchSum['Сумма'].sum().round(2)
# Загрузить минусовые комиссия как плюсы  ####################################
bdfExportNegativeTitleСommission = bdfExportAll.copy(deep=True).query\
                    ('ВидОперации.notna() & Количество < 0', engine='python')
bdfExportNegativeTitleСommission['Количество'] = \
                         bdfExportNegativeTitleСommission['Количество'].abs()
bdfExportNegativeTitleСommission['Сумма'] = \
                         bdfExportNegativeTitleСommission['Сумма'].abs()
# Загрузить минусовые свои как плюсы  ########################################
bdfExportNegativeTitlePurchased = bdfExportAll.copy(deep=True).query\
                    ('ВидОперации.isna() & Количество < 0', engine='python')
bdfExportNegativeTitlePurchased['Количество'] = \
                         bdfExportNegativeTitlePurchased['Количество'].abs()
bdfExportNegativeTitlePurchased['Сумма'] = \
                         bdfExportNegativeTitlePurchased['Сумма'].abs()
# Для списать все в двойном размере, подразделение????####################### 
bdfExportNegativeTitleAllx2 = bdfExportAll.copy(deep=True).query\
                                        ('Количество < 0', engine='python')
bdfExportNegativeTitleAllx2['Количество'] = \
                         bdfExportNegativeTitleAllx2['Количество'].abs() * 2
bdfExportNegativeTitleAllx2['Сумма'] = \
                         bdfExportNegativeTitleAllx2['Сумма'].abs() * 2
# Плюсовые комиссия только положительные  ###################################
bdfExportPositiveTitleСommission = bdfExportAll.copy(deep=True).query\
                    ('ВидОперации.notna() & Количество > 0', engine='python')
# Плюсовые свои только положительные   ######################################
bdfExportPositiveTitlePurchased = bdfExportAll.copy(deep=True).query\
                    ('ВидОперации.isna() & Количество > 0', engine='python')
# Плюсовые комиссия все подряд по модулю  ###################################
bdfExportPositiveTitleСommissionAll = bdfExportAll.copy(deep=True).query\
                                   ('ВидОперации.notna()', engine='python')
bdfExportPositiveTitleСommissionAll['Количество'] = \
                         bdfExportPositiveTitleСommissionAll['Количество'].abs()
bdfExportPositiveTitleСommissionAll['Сумма'] = \
                         bdfExportPositiveTitleСommissionAll['Сумма'].abs()
# Плюсовые свои все подряд по модулю      ###################################
bdfExportPositiveTitlePurchasedAll = bdfExportAll.copy(deep=True).query\
                                   ('ВидОперации.isna()', engine='python')
bdfExportPositiveTitlePurchasedAll['Количество'] = \
                         bdfExportPositiveTitlePurchasedAll['Количество'].abs()
bdfExportPositiveTitlePurchasedAll['Сумма'] = \
                         bdfExportPositiveTitlePurchasedAll['Сумма'].abs()

##############################################################################
output1_rows = bdfExportNegativeTitleСommission.shape[0]
output1_qtt = bdfExportNegativeTitleСommission['Количество'].sum().round(2)
output1_sum = bdfExportNegativeTitleСommission['Сумма'].sum().round(2)

output2_rows = bdfExportNegativeTitlePurchased.shape[0]
output2_qtt = bdfExportNegativeTitlePurchased['Количество'].sum().round(2)
output2_sum = bdfExportNegativeTitlePurchased['Сумма'].sum().round(2)

output3_rows = bdfExportPositiveTitleСommission.shape[0]
output3_qtt = bdfExportPositiveTitleСommission['Количество'].sum().round(2)
output3_sum = bdfExportPositiveTitleСommission['Сумма'].sum().round(2)

output4_rows = bdfExportPositiveTitlePurchased.shape[0]
output4_qtt = bdfExportPositiveTitlePurchased['Количество'].sum().round(2)
output4_sum = bdfExportPositiveTitlePurchased['Сумма'].sum().round(2)

check = pd.DataFrame({'Rows': [input_rows, serv_rows, output_rows,
                      output1_rows, output2_rows, output3_rows, output4_rows],
                     'quantity': [input_qtt, serv_qtt, output_qtt,
                     output1_qtt, output2_qtt, output3_qtt, output4_qtt],
                     'sum': [input_sum, serv_sum, output_sum,
                            output1_sum, output2_sum, output3_sum, output4_sum]
                      },index=['Input', 'Service', 'Output',
                         '-com', '-our', '+com', '+our',       ])

check.loc['Check (rows is OK)'] = check.loc['Output'] -\
                              (check.loc['Input'] - check.loc['Service']) 
check.loc['Check (+-= OK)'] =-check.loc['-com'] - check.loc['-our']\
                             +check.loc['+com'] + check.loc['+our']\
                             -check.loc['Output']                                                                   

In [127]:
MoneyDf = MoneyInputDf.copy(deep=True)
MoneyDf['Поставщик_Имя'] = MoneyDf['Имя партнера']
MoneyDf['Признак Аванса'] =\
    ['ДА' if x < 0 else 'НЕТ' for x in MoneyDf['Общий долг']]
MoneyDf['Договор'] = 'Основной договор'   
MoneyDf['Контрагент'] = MoneyDf['Имя партнера']   
MoneyDf['Сумма'] = MoneyDf['Общий долг'].abs()  
MoneyDf['Сумма (вал)'] = MoneyDf['Сумма'] 
MoneyDf = MoneyDf.merge(suppliersDf['Поставщик_Имя'], how='right')
MoneyDf = MoneyDf.drop_duplicates(subset=['Имя партнера'])
MoneyDf = MoneyDf[MoneyDf['Сумма'] != 0]
tails_cut = 20  #убрать мелкий долг меньше 20р
drorred_suppliers = len(MoneyDf[MoneyDf['Сумма'] <= tails_cut])
MoneyDf = MoneyDf[MoneyDf['Сумма'] > tails_cut]  
MoneyDfExport = MoneyDf[['Признак Аванса', 'Договор',
                         'Контрагент', 'Сумма', 'Сумма (вал)']]

## Блок проверки констситентности
## Если проверка не прошла, вылезет ошибка, блок записи не запустится
Список возможных глюков на входе:
*   Единицы измерения не из списка, хранятся в партиях, нужно базовые в партии перезаписывать чтобы накатился фикс. А они не записываются, база не дает, я в восхищении!
*   Непечатываемые символы из Домино (Code31) в частности Ищется поиском в экселе =НАЙТИ(СИМВОЛ(33);A1)
*   Чистить штрихкоды в 1с те, которые поменялись между карточками, чтобы всю пачку не грузить и дублей не было, пустые тоже



In [145]:
# %whos DataFrame
print('-'*80)
inputLen, outputLen = len(df2), len(SKUexportAll)
if inputLen == outputLen:
    print(f'OK Исходник на входе {inputLen} строк совпадает по длине '
          f'с итоговым {outputLen} строк')
else:
    raise Exception(f'Количество строк на входе и выходе не совпадает на:'
                    f'{inputLen - outputLen} строк')

noSupplier = SKUexportAll['Поставщик'].isna().sum()   
if noSupplier == 0:
    print(f'OK {noSupplier} строк без поставщика')
else:
    raise Exception(f'{noSupplier} строк без поставщика')

noUnit = SKUexportAll['Единица измерения'].isna().sum() 
if noUnit == 0:
    print(f'OK {noUnit} строк без единицы измерения')
else:
    raise Exception(f'{noUnit} строк без единицы измерения')

print ('На входе df2 исходник текущий     строк:',df2.shape[0])
print ('Output BarcodeExport         строк:',BarcodeExport.shape[0])
print ('Output BarcodeExportNew      строк:',BarcodeExportNew.shape[0])
print ('Output BarcodeExpMissed      строк:',BarcodeExportMissed.shape[0])
print ('Output BarcodeExpWasted      строк:',BarcodeExportWasted.shape[0])
print ('Output batchExportAll        строк:',batchExport.shape[0])
print ('Output batchExportNewDelete  строк:',batchExportNewDelete.shape[0])
print ('Output batchExportNewInsert  строк:',batchExportNewInsert.shape[0])
print ('Output batchExportOldDelete  строк:',batchExportOldDelete.shape[0])
print ('Output bdfExport(8)          строк:',bdfExportAll.shape[0])
print ('Output PriceExport           строк:',PriceExport.shape[0])
print ('Output SKUexportAll          строк:',SKUexportAll.shape[0])
print ('Output SKUexportNew          строк:',SKUexportNew.shape[0])
print ('Output SKUexportOld          строк:',SKUexportOld.shape[0])
print ('Output SkuUpdate             строк:',SkuUpdate.shape[0])
print ('Output MoneyDfExport         строк:',MoneyDfExport.shape[0])

if drorred_suppliers != 0:
    print(f'В балансах поставщиков обрезаны данные меньше {tails_cut}р. '
          f'{drorred_suppliers} строк')

print('-'*80)
print ('Проверка выгрузки партий')
print('-'*80)
print(check)
print('-'*80)
CheckQtt = check['quantity'][-2]
CheckSum = check['sum'][-2]
TotalQtt = check['quantity'][-1]
TotalSum = check['sum'][-1]
TotalCheck = CheckQtt + CheckSum + TotalQtt + TotalSum
if TotalCheck == 0:
    print(f'OK Qtt      ', CheckQtt)#Строк больше на совпавшие по отделам запасы
    print(f'OK Sum      ', CheckSum)
    print(f'OK TotalQtt ', TotalQtt)#Строк меньше на количество отрицательных
    print(f'OK TotalSum ', TotalSum)#Это только в чеке, так их столько же
else:
    raise Exception(f'{CheckQtt},{CheckQtt},{TotalQtt},{TotalSum}\
     не совпали строки по партиям')
print('-'*80)
is_equal_barcodes_from_1c_strings_count\
    = BarFrom1c['Штрихкод'].nunique() == BarFrom1c.shape[0]
is_unique_barcodes_from_1c = BarFrom1c['Штрихкод'].is_unique

if  is_equal_barcodes_from_1c_strings_count & is_unique_barcodes_from_1c :   
    print(f'OK BarFrom1c Qtt    strings: ',
          is_equal_barcodes_from_1c_strings_count)
    print(f'OK BarFrom1c unique strings: ', is_unique_barcodes_from_1c)
else:
    print(f'Not OK BarFrom1c Qtt    strings: ',
          is_equal_barcodes_from_1c_strings_count)
    print(f'Not OK BarFrom1c unique strings: ', is_unique_barcodes_from_1c)

if BarcodeExportWasted.shape[0] > 0 :
    print ('Not OK Не совпадают штрихкода из 1с с Домино, удали их, строк: '
                                                  ,BarcodeExportWasted.shape[0])
else: 
    print ('OK Совпали Штрихкода из 1с с Домино: '
                             ,BarcodeExportWasted.shape[0] == 0)  

--------------------------------------------------------------------------------
OK Исходник на входе 33856 строк совпадает по длине с итоговым 33856 строк
OK 0 строк без поставщика
OK 0 строк без единицы измерения
На входе df2 исходник текущий     строк: 33856
Output BarcodeExport         строк: 41474
Output BarcodeExportNew      строк: 0
Output BarcodeExpMissed      строк: 0
Output BarcodeExpWasted      строк: 0
Output batchExportAll        строк: 28545
Output batchExportNewDelete  строк: 7
Output batchExportNewInsert  строк: 922
Output batchExportOldDelete  строк: 0
Output bdfExport(8)          строк: 29053
Output PriceExport           строк: 33856
Output SKUexportAll          строк: 33856
Output SKUexportNew          строк: 0
Output SKUexportOld          строк: 2503
Output SkuUpdate             строк: 7566
Output MoneyDfExport         строк: 72
В балансах поставщиков обрезаны данные меньше 20р. 23 строк
-------------------------------------------------------------------------------

##Блок записи в файлы

In [129]:
# engine менять обязательно, иначе не сохраняет эксельки
# ВЫГРУЖАЕТ фрейм BarcodeExport в BarcodesAll.xlsx  
file_name = 'BarcodesAll'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
BarcodeExport.to_excel(file_name, index=False, header=True\
                       , freeze_panes=(1,1), engine='xlsxwriter') 

In [130]:
# ВЫГРУЖАЕТ фрейм BarcodeExportNew в BarcodesNew.xlsx  
file_name = 'BarcodesNew'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
BarcodeExportNew.to_excel(file_name, index=False, header=True\
                       , freeze_panes=(1,1), engine='xlsxwriter') 

In [131]:
# ВЫГРУЖАЕТ фрейм BarcodeExportMissed в BarcodesMissed.xlsx  
file_name = 'BarcodesMissed'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
BarcodeExportMissed.to_excel(file_name, index=False, header=True\
                       , freeze_panes=(1,1), engine='xlsxwriter')

In [132]:
# ВЫГРУЖАЕТ фрейм BarcodeExportWasted в BarcodeWasted.xlsx  
file_name = 'BarcodesWasted'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
BarcodeExportWasted.to_excel(file_name, index=False, header=True\
                       , freeze_panes=(1,1), engine='xlsxwriter')

In [133]:
# ВЫГРУЖАЕТ фрейм PriceExport в Price.xlsx  
file_name = 'Price'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
PriceExport.to_excel(file_name, index=False, header=True\
                     , freeze_panes=(1,1), engine='xlsxwriter') 

In [134]:
# Выгружает Фрейм SKUexportAll в SkuAll.xlsx 
# Справочник всех товаров из исходника для первичного импорта в 1С
file_name = 'SkuAll'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
SKUexportAll.to_excel(file_name, index=False, header=True\
                      , freeze_panes=(1,1), engine='xlsxwriter')

In [135]:
# Выгружает Фрейм SKUexportNew в SkuNew.xlsx 
# Справочник новых товаров из исходника для импорта в 1С
file_name = 'SkuNew'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
SKUexportNew.to_excel(file_name, index=False, header=True\
                      , freeze_panes=(1,1), engine='xlsxwriter')

In [136]:
# Выгружает Фрейм SKUexportOld в SkuOld.xlsx 
# Справочник новых товаров из исходника для импорта в 1С
file_name = 'SkuOld'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
SKUexportOld.to_excel(file_name, index=False, header=True\
                      , freeze_panes=(1,1), engine='xlsxwriter')

In [137]:
# Выгружает Фрейм SkuUpdate в SkuUpdate.xlsx 
# Справочник новых товаров из исходника для импорта в 1С
file_name = 'SkuUpdate'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
SkuUpdate.to_excel(file_name, index=False, header=True\
                      , freeze_panes=(1,1), engine='xlsxwriter')

In [138]:
# Выгружает Фрейм batchExport в batchExportAll.xlsx
# Справочник партий, xlsxwriter партии жрет как числа, лечится .astype(str)
file_name = 'batchExportAll'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
batchExport.to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')

In [139]:
# Выгружает Фрейм batchExportNewInsert в batchExportNewInsert.xlsx
# Справочник партий, xlsxwriter партии жрет как числа, лечится .astype(str)
file_name = 'batchExportNewInsert'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
batchExportNewInsert.to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')

In [140]:
# Выгружает Фрейм batchExportNewDelete в batchExportNewDelete.xlsx
# Справочник партий, xlsxwriter партии жрет как числа, лечится .astype(str)
file_name = 'batchExportNewDelete'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
batchExportNewDelete.to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')

In [141]:
# Выгружает Фрейм batchExportOldDeletee в batchExportOldDelete.xlsx
# Справочник партий, xlsxwriter партии жрет как числа, лечится .astype(str)
file_name = 'batchExportOldDelete'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
batchExportOldDelete.to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')

In [142]:
# Выгружает # Фреймы bdfExport в Export.xlsx
# остатки по партиям
dfList = ['bdfExportAll', 'bdfExportNegativeTitlePurchased',
   'bdfExportNegativeTitleСommission', 'bdfExportPositiveTitlePurchased',
   'bdfExportPositiveTitleСommission', 'bdfExportNegativeTitleAllx2',
   'bdfExportPositiveTitleСommissionAll', 'bdfExportPositiveTitlePurchasedAll']
fileList =  [s.replace('bdfExport','Export') for s in dfList]

i = 0
for x in dfList:
    file_name = fileList[i]
    file_name = ''.join((output_path, file_name, time_now, to_xlsx))
    locals()[x].to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')
    i = i + 1

In [143]:
# Выгружает Фрейм MoneyDfExport в Suppliers_Balance.xlsx
file_name = 'Suppliers_Balance'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
MoneyDfExport.to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')

 Шпаргалка

In [144]:
#@title ШПАРГАЛКА

# df.columns
# df.info()
# df.dtypes
# df.describe()
# df['ТОВАР'] #одна колонка
# df[['ТОВАР','1эт остаток','2эт остаток']] #3 колонки
# df['ТОВАР'].hist #гистограмма
# df['SubGroupe'].value_counts() # подсчет уникальных значений
# df['SubGroupe'].value_counts(dropna=False) #включая пустые
# q = df['Продано'].quantile(0.25)
# df[df['Продано'] < q]
# BarcodeExport[:10].to_csv #первые 10
# df[df2['1эт остаток'].notna()]
# df = skuDf[skuDf['Счет учета затрат'].isna()] 
# df = skuDf[skuDf['Вид маркируемой продукции ИС МП'].notna()] 
# df['SubGroupe'].value_counts(dropna=False)
# skuDf.groupby('В группе',dropna=False)['РозницаРуб'].agg(['count','sum','mean']).sort_values(by='sum', ascending=False)
# BarcodeExport['ТОВАР'] = BarcodeExport['ТОВАР'].astype('Int64') 
# batch['ТОВАР'] = batch['Партия'].astype('Float64').astype('Int64') #перевод партии сначала из текста с точкой в флоат, потом в инт
# df3=df1.loc[~df['a'].isin(df2['a'])] #вычесть строки одной из другой тоже способ
# df[df["col"]. str.contains (" this string ")] поиск в базе 'unicode_escape'
# BarcodeExport['Номенклатура'] = 'ДМ-' + BarcodeExport['ТОВАР'].apply('{:0>8} '. format).astype (str) #приклеить префикс
# bdf.groupby('ГРУППА',dropna=False)['ИСХ_КОЛИЧ'].agg(['count','sum']).sort_values(by='ГРУППА', ascending=True)
# Таблица не мерджится тк в 1с всосали все подгруппы в верхнем регистре
# df_address['country_lower'] = df_address['Country'].str.lower()

# ??pd.DataFrame.to_excel
# find = df2[df2['НАИМЕНОВАНИЕ'].str.contains('| '.join('unicode_escape'))]
# find = skuDf[skuDf['НАИМЕНОВАНИЕ'].str.contains('САМОРЕЗ')]
# ??pd.DataFrame.to_excel
# df1 = df1.replace('unicode_escape',' ', regex=True) # В Теории это лечит непечатные символы, ток непонятно, работает ли. Не работает
# !pip install xlsxwriter #for xlsxwriter as engine
# print ('Я дошёл до конца и выполнился')

#BarcodeExportWasted = BarcodeExportWasted.query('Номенклатура.Наименование.str.upper.contains("!ТЕСТ")', engine='python') #нихт фунциклирен возможно из-за точки в имени колонки
# df[df['A'].str.contains("hello")]
# df1.query('col.str.contains("foo")', engine='python')
# df[df['A'].str.contains("Hello|Britain")==True]
# batchExportOldDelete.groupby(['_merge'],dropna=False)['_merge'].count()